# Aguirregabiria and Mira (2010)
The types of models covered by StructuralEconometrics.jl can also be used to model
entry and exit decisions of firms. Below we present Example 4 in \cite{AM2010}, but
without the strategic interaction between firms. Estimation of dynamic games is not
yet implemented in this package. Since this paper provides a solid, but approachable,
introduction to the methods applied in our package, we repeat a version of their example
here.

As each period begins, the firms observe two things: if they entered (or stayed) in the market last period or left (or stayed out), $a_{t-1}\in\{ 0,1\}$,
and a market demand state $M\in\{1,2,3,4,5\}$. If they are not yet active in the market, they will
face entry costs if they choose to enter. If they are already active in the market,
they will only have to pay the fixed costs of operating in the market. The market
demand state shifts revenue up or down, such that it is less attractive to enter
a market with low activity compared to one with lots of activity. Payoff given
non-market participation is normalized to zero, and payoff given participation is
\begin{align}
    U(enter|M_t, a_{t-1}) = \theta_{RS}\log(M_t)-\theta_{FC}-\theta_{EC}(1-a_{t-1})
\end{align}
or
\begin{align}
    U(enter|M_t, a_{t-1}) = (1; log(M_t); a_{t-1}-1)'\theta,
\end{align}
Given a transition matrix for $M$, this is all the information we need to set up
this model. The discount factor is set to 0.95, and the parameter vector is $\theta=(-1, 1, 2)$.

In [1]:
using StocasEstimators

In [2]:
bdisc = 0.95    # discount factor

# Parameters in the data generating process
truepar = [-1.; # fixed cost parameter
            1.; # revenue parameter
            2.] # entry cost parameter
# State 1: Exogenous state
nX1 = 2    #
X1 = ["not present"; "present"] # incumbancy status
F1 = [sparse([1. 0; 1 0]), # firm will stay out or leave the market
      sparse([0. 1; 0 1])] # firm will enter or stay in the market
S1 = State("incumbancy", X1, F1)

# State 2 Exogenous market-wide state
nX2 = 5
X2 = 1:nX2 # state variable levels 1 through nX2

M = sparse([0.8 0.2 0.0 0.0 0.0;
            0.2 0.6 0.2 0.0 0.0;
            0.0 0.2 0.6 0.2 0.0;
            0.0 0.0 0.2 0.6 0.2;
            0.0 0.0 0.0 0.2 0.8;])

S2 = CommonState("market demand", X2, M)

# Collect states to one States instance
S = States(S1, S2)

# Create choice specific utility variable matrices
Z1 = zeros(nX2*2, 3) # "out of market" payoff normalized to zero
Z2 = [-ones(nX2) log.(X2) -ones(nX2);  # Z(entry) if firm was not in the market
      -ones(nX2) log.(X2)  zeros(nX2)] # Z(entry) if firm was already in the market
U = LinearUtility(("don't enter", "enter"), (Z1, Z2), bdisc, truepar)

Stocas.LinearUtility{2,Float64}(("don't enter", "enter"), ([0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], [-1.0 0.0 -1.0; -1.0 0.693147 -1.0; … ; -1.0 1.38629 0.0; -1.0 1.60944 0.0]), 0.95, [-1.0, 1.0, 2.0], ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.0, -0.306853, 0.0986123, 0.386294, 0.609438, 1.0, 1.69315, 2.09861, 2.38629, 2.60944]), ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), 3)

Again, we can simply plot the payoffs using the $\texttt{plot(U)}$ function.

In [3]:
Vvfi, itersvfi = solve(U, S, Stocas.VFI())
Vnewt, itersnewt = solve(U, S, Stocas.Newton())
Vpoli, iterspoli = solve(U, S, Stocas.Policy())
Vpoly, iterspoly = solve(U, S, Stocas.Poly())

(Stocas.IntegratedValueFunction{Array{Float64,1},Array{Float64,2},Array{Array{Float64,1},1}}([31.7835, 34.7474, 38.3617, 41.4472, 43.3178, 33.4125, 36.5585, 40.2397, 43.3569, 45.2458], [31.7835, 34.7474, 38.3617, 41.4472, 43.3178, 33.4125, 36.5585, 40.2397, 43.3569, 45.2458], [0.272409 0.0681022 … 0.0 0.0; 0.0378336 0.113501 … 0.0 0.0; … ; 0.0 0.0 … 0.561561 0.187187; 0.0 0.0 … 0.18778 0.751118], Array{Float64,1}[[30.7574, 33.1336, 36.3431, 39.144, 40.7965, 30.7574, 33.1336, 36.3431, 39.144, 40.7965], [32.3396, 34.8323, 38.1205, 40.9557, 42.6246, 32.3396, 34.8323, 38.1205, 40.9557, 42.6246]]), [12, 2])

In [4]:
M, N, T = 1, 5300, 1000
ds = simulate(U, S, M, N, T)

# Estimate transitions
TN = T*N
F2_est = zeros(nX2, nX2)
for n = 1:T*N-1
    if n%T != 0
        F2_est[ds.xs[n, 2], ds.xs[n + 1, 2]] += 1
    end
end

F2_est = F2_est./sum(F2_est,2)
S_est =  States(State(X1, F1), CommonState(X2, F2_est))

Discrete states
 * Number of state variables: 2


In [5]:
result_nfxp = fit_nfxp(U, S_est, ds)
result_npl = fit_npl(U, S_est, ds)
display(result_nfxp), println(), display(result_npl);

 * Total number of states:    10
 * Individual states:           
   1) state (n: 2)
   2) common state (n: 5)
Results of estimation
 * Method: NFXP
 * loglikelihood: -661050.889872363


 * Estimate: [-0.9995737232057226,0.9970499613512077, ...]
 * Std.err.: [0.004245089679969354,0.003996288686787287, ...]
 Iterations
 * Maximum likelihood: 2
 * Contractions: 59
 * Newton steps: 8

Results of estimation


 * Method: NPL
 * loglikelihood: -661050.8899681416
 * Estimate: [-0.999573726449975,0.9970498793722483,1.999588060222941]
 * Std.err.: [0.004242963761790469,0.003981528879125675,0.006727944072552153]
 Iterations
 * Policy iteration: 3
 * Maximum likelihood: 3


In [6]:
using Plots
pgfplots()
plot(("not present", :), S, U, :policy, title="Entry probabilities for different market demand.")

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 Z M 2.9375 -6.375 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.125 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 L 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 Z M 1.265625 -0.765625 "/>
 
 
<path style="stroke:none;" d="M 2.890625 -3.515625 C 3.703125 -3.78125 4.28125 -4.46875 4.28125 -5.265625 C 4.28125 -6.078125 3.40625 -6.640625 2.453125 -6.640625 C 1.453125 -6.640625 0.6875 -6.046875 0.6875 -5.28125 C 0.6875 -4.953125 0.90625 -4.765625 1.203125 -4.765625 C 1.5 -4.765625 1.703125 -4.984375 1.703125 -5.28125 C 1.703125 -5.765625 1.234375 -5.765625 1.09375 -5.765625 C 1.390625 -6.265625 2.046875 -6.390625 2.40625 -6.390625 C 2.828125 -6.390625 3.375 -6.171875 3.375 -5.28125 C 3.375 -5.15625 3.34375 -4.578125 3.09375 -4.140625 C 2.796875 -3.65625 2.453125 -3.625 2.203125 -3.625 C 2.125 -3.609375 1.890625 -3.59375 1.8125 -3.59375 C 1.734375 -3.578125 1.671875 -3.5625 1.671875 -3.46875 C 1.671875 -3.359375 1.734375 -3.359375 1.90625 -3.359375 L 2.34375 -3.359375 C 3.15625 -3.359375 3.53125 -2.6875 3.53125 -1.703125 C 3.53125 -0.34375 2.84375 -0.0625 2.40625 -0.0625 C 1.96875 -0.0625 1.21875 -0.234375 0.875 -0.8125 C 1.21875 -0.765625 1.53125 -0.984375 1.53125 -1.359375 C 1.53125 -1.71875 1.265625 -1.921875 0.984375 -1.921875 C 0.734375 -1.921875 0.421875 -1.78125 0.421875 -1.34375 C 0.421875 -0.4375 1.34375 0.21875 2.4375 0.21875 C 3.65625 0.21875 4.5625 -0.6875 4.5625 -1.703125 C 4.5625 -2.515625 3.921875 -3.296875 2.890625 -3.515625 Z M